In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

import os
import sys
import time
import numpy as np
import math
import json
from tqdm import tqdm_notebook as tqdm
import pathlib
from pathlib import Path
import time

import matplotlib.pyplot as plt 
%matplotlib inline

import torch

import deep_sdf
import deep_sdf.workspace as ws
from models import *
from datasets import *
from custom_utils import *

from sklearn.neighbors import KDTree

import torch
import numpy
import trimesh
import plyfile
import numpy as np
import numbers
import torch
from torch import nn
from torch.nn import functional as F
import math

def computeAvgTransform():
    objects = list()
    for (dirpath, dirnames, filenames) in os.walk("/cvlabdata2/home/artem/Data/cars_remeshed_dsdf/transforms/"):
        objects += [os.path.join(dirpath, file) for file in filenames if file[-4:] == '.npy']
    
    matricies = []
    for obj in objects:
        matricies.append(np.load(obj))
    
    return np.mean(np.array(matricies), axis=0)

AvgTransform = computeAvgTransform()

def transformPoints(points, matrix):
    matrix = torch.cuda.FloatTensor(matrix)
    column = torch.zeros((len(points), 1), device="cuda:0") + 1
    stacked = torch.cat([points, column], dim=1)
    transformed = torch.matmul(matrix, stacked.T).T[:, :3]
    return transformed

def make_mesh_from_points(points, ply_mesh):
    transformed_points = transformPoints(points, AvgTransform)
    
    edges = trimesh.geometry.faces_to_edges(ply_mesh['face']['vertex_indices'])
    np_points = transformed_points.cpu().detach().numpy()
    edge_attr = [np_points[a] - np_points[b] for a, b in edges]

    data = torch_geometric.data.Data(x  = transformed_points, 
                                     pos= transformed_points, 
                                     face = torch.tensor(ply_mesh['face']['vertex_indices'], 
                                                         dtype=torch.long).to('cuda:0').t(),
                                     edge_attr=torch.tensor(edge_attr, dtype=torch.float).to('cuda:0'),
                                     edge_index=torch.tensor(edges, dtype=torch.long).t().contiguous().to('cuda:0'))
    return data

def boundsLoss(points, box=[(-1, 1, 0)]):
    loss = 0
    for l, r, i in box:
        loss +=  torch.mean(F.relu(-points[:, i] + l))  \
               + torch.mean(F.relu( points[:, i] - r))
    return loss

def innerBoundsLoss(points, r=1, center=(0, 0, 0)):
    radiuses = torch.sum( (points - torch.Tensor(center).to('cuda:0')) ** 2 , dim=1)
    return torch.sum(F.relu(r - radiuses))

def calculate_loss(mesh, local_preds, axis=0, constraint_rad=0.1):
    loss =  (1 - axis) * compute_lift_faces_diff(mesh, local_preds, axis=0) + \
                  axis * compute_lift_faces_diff(mesh, local_preds, axis=1)
    
    loss += boundsLoss(mesh.x, box=[(-0.6, 0.6, 0)])
    loss += innerBoundsLoss(mesh.x, r=(constraint_rad / 2)**2, center=(0.05, 0.05, 0))  \
          + innerBoundsLoss(mesh.x, r=(constraint_rad / 2)**2, center=(-0.3, 0, 0))

    return loss

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
experiment_directory = "/cvlabdata2/home/artem/DeepSDF/examples/cars_cleared/"
checkpoint = "latest"
decoder = load_model(experiment_directory, checkpoint).to('cuda:0')

latent_vectors = ws.load_latent_vectors(experiment_directory, checkpoint)
latent_size = latent_vectors[0].size()[0]
print(f"{len(latent_vectors)} of latent vectors, each {latent_size} long")

1205 of latent vectors, each 1 long


In [4]:
def trilinear_interpolate(grid, samples):
    """
    Interpolate 3D samples on cartesian grid of values.
    Note that the code assumes samples bounding box to 
    be alligned to cartesian grid.
    
    input:
    image \in [B,C,W,W,W]
    samples \in [B,N,3]

    output:
    interpolated_samples \in [B,N,C]
    
    """

    W = grid.shape[-1]
    samples = samples.unsqueeze(2).unsqueeze(3).unsqueeze(3)
    samples = (samples[:,:,:,0]/(W-1)) # normalize to between  0 and 1
    samples = samples*2-1 # normalize to between -1 and 1
    interpolated_samples = torch.nn.functional.grid_sample(grid, samples)
    return interpolated_samples.squeeze(-1).squeeze(-1).permute(0,2,1)

def write_verts_faces_to_file(verts, faces, ply_filename_out):

    num_verts = verts.shape[0]
    num_faces = faces.shape[0]

    verts_tuple = np.zeros((num_verts,), dtype=[("x", "f4"), ("y", "f4"), ("z", "f4")])

    for i in range(0, num_verts):
        verts_tuple[i] = tuple(verts[i, :])

    faces_building = []
    for i in range(0, num_faces):
        faces_building.append(((faces[i, :].tolist(),)))
    faces_tuple = np.array(faces_building, dtype=[("vertex_indices", "i4", (3,))])

    el_verts = plyfile.PlyElement.describe(verts_tuple, "vertex")
    el_faces = plyfile.PlyElement.describe(faces_tuple, "face")

    ply_data = plyfile.PlyData([el_verts, el_faces])
    ply_data.write(ply_filename_out)
    

class GaussianSmoothing(nn.Module):
    """
    Apply gaussian smoothing on a
    1d, 2d or 3d tensor. Filtering is performed seperately for each channel
    in the input using a depthwise convolution.
    Arguments:
        channels (int, sequence): Number of channels of the input tensors. Output will
            have this number of channels as well.
        kernel_size (int, sequence): Size of the gaussian kernel.
        sigma (float, sequence): Standard deviation of the gaussian kernel.
        dim (int, optional): The number of dimensions of the data.
            Default value is 2 (spatial).
    """
    def __init__(self, channels, kernel_size, sigma, dim=3):
        super(GaussianSmoothing, self).__init__()
        if isinstance(kernel_size, numbers.Number):
            kernel_size = [kernel_size] * dim
        if isinstance(sigma, numbers.Number):
            sigma = [sigma] * dim

        # The gaussian kernel is the product of the
        # gaussian function of each dimension.
        kernel = 1
        meshgrids = torch.meshgrid(
            [
                torch.arange(size, dtype=torch.float32)
                for size in kernel_size
            ]
        )
        for size, std, mgrid in zip(kernel_size, sigma, meshgrids):
            mean = (size - 1) / 2
            kernel *= 1 / (std * math.sqrt(2 * math.pi)) * \
                      torch.exp(-((mgrid - mean) / (2 * std)) ** 2)

        # Make sure sum of values in gaussian kernel equals 1.
        kernel = kernel / torch.sum(kernel)

        # Reshape to depthwise convolutional weight
        kernel = kernel.view(1, 1, *kernel.size())
        kernel = kernel.repeat(channels, *[1] * (kernel.dim() - 1))

        self.register_buffer('weight', kernel)
        self.groups = channels

        if dim == 1:
            self.conv = F.conv1d
        elif dim == 2:
            self.conv = F.conv2d
        elif dim == 3:
            self.conv = F.conv3d
        else:
            raise RuntimeError(
                'Only 1, 2 and 3 dimensions are supported. Received {}.'.format(dim)
            )

    def forward(self, input):
        """
        Apply gaussian filter to input.
        Arguments:
            input (torch.Tensor): Input to apply gaussian filter on.
        Returns:
            filtered (torch.Tensor): Filtered output.
        """
        return self.conv(input, weight=self.weight, groups=self.groups)

In [5]:
def apply_parameterization(mesh, displacement_grid,  GRID_RESOLUTION=8, padding=0.2):
    answ = mesh.clone()
    # first go from DeepSDF bounding box [-1,1]^3 to GRID [0,GRID_RESOLUTION]^3
    shifts = torch.min(answ.x, axis=0).values
    scales = (torch.max(answ.x, axis=0).values - torch.min(answ.x, axis=0).values)
    verticies_0_1 = (answ.x - shifts) / scales
                    
    vertices_in_grid = GRID_RESOLUTION*(verticies_0_1 + padding) / (1 + 2 * padding)
    # now read up displacement 
    vertices_displacement = trilinear_interpolate(displacement_grid.unsqueeze(0), 
                                                  vertices_in_grid.unsqueeze(0)).squeeze(0)
    # and sum it to original position
    vertices_displaced = vertices_displacement + vertices_in_grid
    
    # finally go back to DeepSDF bounding box
    answ.x = (vertices_displaced / GRID_RESOLUTION * (1 + 2 * padding) - padding)  * scales + shifts
    answ.pos = answ.x
#     attrs = (answ.x[ mesh.edge_index[0] ] - answ.x[mesh.edge_index[1]]).detach().cpu().numpy()
#     answ.edge_attr = torch.tensor(attrs, dtype=torch.float).to('cuda:0')

#     print(torch.max( torch.sum((mesh.edge_attr -  answ.edge_attr)**2, axis=1))) 
    return answ
   

In [6]:
def optimize_shape_as_umetami(model, inp, num_iters=100, save_to_dir='Expirements/Optimization',
                   lr=0.05, decreased_by=2, adjust_lr_every=10, verbose=None, constraint_rad=0.1, 
                              GRID_RESOLUTION = 8,axis=0):
    
    def adjust_learning_rate(
        initial_lr, optimizer, num_iterations, decreased_by, adjust_lr_every
    ):
        lr = initial_lr * ((1 / decreased_by) ** (num_iterations // adjust_lr_every))
        for param_group in optimizer.param_groups:
            param_group["lr"] = lr
            
        return lr

    if not os.path.exists(os.path.join(save_to_dir, 'meshes')):
        os.makedirs(os.path.join(save_to_dir, 'meshes'))
    if not os.path.exists(os.path.join(save_to_dir, 'predictions')):
        os.makedirs(os.path.join(save_to_dir, 'predictions'))

    model.eval()
    data_instance = inp.clone()
#     data_instance.x.requires_grad = True
    
    
    params = torch.zeros(( 3, GRID_RESOLUTION, GRID_RESOLUTION, GRID_RESOLUTION), 
                            dtype=torch.float).to('cuda:0')
    params.requires_grad = True

    optimizer = torch.optim.SGD([params], lr=lr)

    meshes = []
    lifts = []
    lr_plot = []

    for i in range(num_iters):
        cur_rl = adjust_learning_rate(lr, optimizer, i, decreased_by, adjust_lr_every)
        save_path = os.path.join(save_to_dir, 'meshes/' + str(i).zfill(5) + ".ply")
        preds_save_path = os.path.join(save_to_dir, 'predictions/' + str(i).zfill(5) + ".npy")

        optimizer.zero_grad()
        
        # Get displacement grid
        p3d = (2, 2, 2, 2, 2, 2) 
        displacement_grid = F.pad(params, p3d, "constant", 0)
        smoothing = GaussianSmoothing(3, 5, 1).to('cuda:0')
        displacement_grid = smoothing(displacement_grid.unsqueeze(0)).squeeze(0)

        # we also probably want the displacement to be limited to the dsize of a voxel cell
        displacement_grid = 0.5*torch.tanh(displacement_grid)
        
        
        # transform mesh
        transformed_mesh = apply_parameterization(data_instance, displacement_grid, GRID_RESOLUTION=GRID_RESOLUTION)

        local_preds = model(transformed_mesh)
        loss = calculate_loss(transformed_mesh, local_preds, axis=axis, constraint_rad=constraint_rad)
        loss.backward()
        optimizer.step()

        if verbose is not None and i % verbose == 0:
            print('Iter %4d ' % i, 'Loss: %0.5f ' % loss.detach().cpu().numpy(), ' LR: %0.5f ' % cur_rl, 
                  'Params norm: %0.3f ' % torch.sum(displacement_grid ** 2).detach().cpu().numpy(),
                  'Number params affected: %d ' % torch.sum(torch.sum(displacement_grid**2, axis=0) > 0.05).detach().cpu().numpy())
                #plot_points_from_torch(points)

        tri_mesh = get_trimesh_from_torch_geo_with_colors(transformed_mesh, local_preds)
        tri_mesh.export(save_path)
        np.save(preds_save_path, local_preds.cpu().detach().numpy())

        lifts.append(loss.detach().cpu().numpy())
        lr_plot.append(cur_rl)

        np.save(os.path.join(save_to_dir, "loss_plot.npy"), lifts)
        np.save(os.path.join(save_to_dir, "lr_plot.npy"), lr_plot)

In [7]:
device = "cuda:0"

#model = SplineCNN8(3, batchnorm1=False)
#model.load_state_dict(torch.load("Expirements/SplineCNN8.nn"))
model = SplineCNN8Residuals(3)
model.load_state_dict(torch.load("Expirements/Networks15/normilized_full_latest.nn"))
model = model.to(device)
model = model.eval()

In [10]:
def calculate_loss(mesh, local_preds, axis=0, constraint_rad=0.1):
    loss =  (1 - axis) * compute_lift_faces_diff(mesh, local_preds, axis=0) + \
                  axis * compute_lift_faces_diff(mesh, local_preds, axis=1)
    loss += boundsLoss(mesh.x, box=[(-0.6, 0.6, 0)])
    loss += innerBoundsLoss(mesh.x, r=(constraint_rad / 2)**2, center=(0.05, 0.05, 0))  \
          + innerBoundsLoss(mesh.x, r=(constraint_rad / 2)**2, center=(-0.3, 0, 0))
    return loss

In [11]:
N=256

for LETENT_IDX in [535, 113, 69]:
    LATENT_TO_OPTIMIZE = latent_vectors[LETENT_IDX]
    LATENT_KD_TREE = KDTree(np.array([lv.cpu().detach().numpy()[0] for lv in latent_vectors]))

    with torch.no_grad():
        ply_mesh = create_mesh( decoder,
                                LATENT_TO_OPTIMIZE,
                                N=N,
                                max_batch=int(2 ** 18),
                                offset=None,
                                scale=None)

    points = torch.cuda.FloatTensor(np.hstack(( ply_mesh['vertex']['x'][:, None], 
                                            ply_mesh['vertex']['y'][:, None], 
                                            ply_mesh['vertex']['z'][:, None])))

    MESH_TO_OPTIMIZE = make_mesh_from_points(points, ply_mesh)
    MESH_TO_OPTIMIZE.to('cuda:0')
    
    optimize_shape_as_umetami(model, MESH_TO_OPTIMIZE, verbose=25, lr=1e3, axis=0, GRID_RESOLUTION = 8, num_iters=30,
                          save_to_dir='Expirements/OptimizationPaper/AfterMeeting/UmetamiDrag2/%04dplus' % LETENT_IDX)

Iter    0  Loss: 0.05752   LR: 1000.00000  Params norm: 0.000  Number params affected: 0 
Iter   25  Loss: -0.05255   LR: 250.00000  Params norm: 67.130  Number params affected: 346 
Iter    0  Loss: 0.07473   LR: 1000.00000  Params norm: 0.000  Number params affected: 0 
Iter   25  Loss: -0.03357   LR: 250.00000  Params norm: 52.033  Number params affected: 303 
Iter    0  Loss: 0.11710   LR: 1000.00000  Params norm: 0.000  Number params affected: 0 
Iter   25  Loss: -0.01405   LR: 250.00000  Params norm: 69.313  Number params affected: 394 


In [21]:
GRID_RESOLUTION=8
params = torch.zeros(( 3, GRID_RESOLUTION, GRID_RESOLUTION, GRID_RESOLUTION), 
                            dtype=torch.float).to('cuda:0')
params.requires_grad = True
# Get displacement grid
p3d = (2, 2, 2, 2, 2, 2) 
displacement_grid = F.pad(params, p3d, "constant", 0)
smoothing = GaussianSmoothing(3, 5, 1).to('cuda:0')
displacement_grid = smoothing(displacement_grid.unsqueeze(0)).squeeze(0)

# we also probably want the displacement to be limited to the dsize of a voxel cell
displacement_grid = 0.5*torch.tanh(displacement_grid)